In [ ]:
import sys, os

#change the path to the directory that ctmc.py is located in on your system
sys.path.append(os.path.expanduser('~/source/discrete_states/'))

In [ ]:

from ctmc import ContinuousTimeMarkovChain as MC
from ctmc import normal_generator, gamma_generator, uniform_generator, cyclic_generator, detailed_balance_generator

import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
# example of doing a sweep over many different kind of generators with different S and N, saving the output. The NESS calculating will become numeric if the matrices get
# too large to invert exactly
SS = [3, 6, 20, 100, 250]
trials = [500, 500, 500, 500, 500]

normal_output, uniform_output, gamma_output, cyclic_output, balanced_output = {}, {}, {}, {}, {}

for a in range(5):
    print(a)
    output = [normal_output, uniform_output, gamma_output, cyclic_output, balanced_output ][a]
    gen = [normal_generator, uniform_generator, gamma_generator, cyclic_generator, detailed_balance_generator][a]
    
    
    for s,trial in zip(SS,trials):
        p = MC(S=s, N=trial, generator=gen)
        #R = [ np.random.uniform(0,1,(S,S)) for i in range(trials)]
        #for p,r in zip(procs,R):
        #    p.set_rate_matrix(r, max_rate=1)   
        dct = {'ness':p.get_epr(p.get_ness()), 
               'meps':p.get_epr(p.get_meps()), 
               'unif':p.get_epr(p.get_uniform()),
               'rand':p.get_epr(p.get_random_state()),
               'local':p.get_epr(p.get_local_state()),
               'N':trial, 'scale':p.scale, 
               'nm_dkl':p.dkl(p.ness,p.meps),'mn_dkl':p.dkl(p.meps,p.ness)}
        
        
        output[f'{p.S}'] = dct
        


In [ ]:
# the rest is just some random plot examples

In [ ]:
# just plotting some stuff
fig, ax = plt.subplots(1, len(SS), figsize=(20,8), sharey=True, sharex=True)

data = [normal_output, uniform_output, gamma_output, cyclic_output, balanced_output ]
color = ['b', 'orange', 'green', 'r', 'k']

op = .5

for color, data in zip(color,data):
    for a,k in zip(ax,data.keys()):
        epr = data[k]
        
        ness_diff = (epr['local']-epr['meps'])
        unif_diff = (epr['ness']-epr['meps'])
        
        
        a.scatter(epr['meps'],ness_diff, c=color, alpha=op)
        a.scatter(epr['meps'],unif_diff, marker='x', c=color, alpha=op)
        
        a.axhline(np.mean(ness_diff), c=color)
        a.axhline(np.mean(unif_diff), c=color, linestyle='--')
        
        a.set_title(f'S={k}, N={epr["N"]}')
        
    ax[0].set_yscale('log')
    ax[0].set_ylabel('$\\Sigma_R-\\Sigma_R(MEPS)$')
    ax[0].set_xlabel('$\\Sigma_R(MEPS)$')
    ax[0].set_xlim(1E-4,.7)
    ax[0].set_xscale('log')
    
fig.suptitle('x:ness, o:local, blue:normal dist, orange:uniform dist, green:gamma dist')

In [ ]:
output=cyclic_output

fig, ax = plt.subplots(2, figsize=(10,10))
for k in list(output.keys())[:]:
    epr = output[k]
    ness_diff = np.mean(   (epr['ness']-epr['meps'])/epr['meps'] )
    unif_diff = np.mean(  (epr['unif']-epr['meps'])/epr['meps'] )
    ax[1].scatter(k,ness_diff, c='tab:orange')
    ax[1].scatter(k,unif_diff, c='tab:blue')
    
    ness_diff = np.mean(   (epr['ness']-epr['meps']) )
    unif_diff = np.mean(  (epr['unif']-epr['meps']) )
    ax[0].scatter(k,ness_diff, c='tab:orange')
    ax[0].scatter(k,unif_diff, c='tab:blue')


    
ax[0].set_ylabel('$\\langle \\Sigma_R - \\Sigma_R(MEPS) \\rangle$')

ax[1].set_ylabel('$\\langle \\left(\\Sigma_R - \\Sigma_R(MEPS)\\right)/(\\Sigma_R(MEPS)) \\rangle$')

ax[0].set_xlabel('$|S|$')
ax[1].set_xlabel('$|S|$')

ax[1].set_yscale('log')

In [ ]:
dbfig, ax = plt.subplots(3, figsize=(8,8), sharey=True, sharex=True )
names = ['random state', 'local state', 'min ent state']

sizes = [5,20,100, 250]
N=500

for i,size in enumerate(sizes):
    pr = MC(S=size, N=N, generator=detailed_balance_generator)
    for k, get_state in enumerate([pr.get_random_state, pr.get_local_state, pr.get_meps]):
        s = get_state()
        j = pr.get_prob_current(s)
        ax[k].scatter(size*np.ones(N), -j**2*pr.get_activity(s)+pr.get_epr(s), alpha=.2)

for a,n in zip(ax,names):
    a.set_title(n)
ax[0].set_yscale('log')  
ax[0].set_xscale('log') 

ax[-1].set_xlabel('|S|')
ax[1].set_ylabel('short time speed limit delta')

In [ ]:
cfig, ax = plt.subplots(3, figsize=(8,8), sharey=True, sharex=True )
names = ['random state', 'local state', 'min ent state']

sizes = [5,20,100, 250]
N=500

for i,size in enumerate(sizes):
    pr = MC(S=size, N=N, generator=cyclic_generator)
    for k, get_state in enumerate([pr.get_random_state, pr.get_local_state, pr.get_meps]):
        s = get_state()
        j = pr.get_prob_current(s)
        ax[k].scatter(size*np.ones(N), -j**2*pr.get_activity(s)+pr.get_epr(s), alpha=.2)

for a,n in zip(ax,names):
    a.set_title(n)
ax[0].set_yscale('log')  
ax[0].set_xscale('log') 

ax[-1].set_xlabel('|S|')
ax[1].set_ylabel('short time speed limit delta')


In [ ]:
pr = MC(N=1, S=7, generator=cyclic_generator)
s = pr.get_local_state(sigma=np.array([1]))

In [ ]:
L=100
st = np.empty(shape=(L,pr.S))

i=0
dt=.1
while i<L:
    s += dt*np.squeeze((pr.get_time_deriv(s)))
    st[i] = s
    i += 1

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,8), sharex=True)
st = np.array(st)[:50]
ax[0].plot(st);
ax[1].plot(pr.get_activity(st),'b', label='activity')

j = pr.get_prob_current(s)

ax[1].plot(j**2 * pr.get_activity(st), label='bound')
eprs = []
for item  in st:
    eprs.append(pr.get_epr(item))
eprs = np.array(eprs)

ax[1].plot(eprs,'k', label='epr')
fig.legend()

#ax[1].set_yscale('log')
ax[0].set_title('state dynamics')
ax[1].set_title('epr metrics')

ax[0].set_xlabel('t')
